In [1]:
import numpy as np 
import os 
import sys 
import xarray as xr
import pandas as pd

path = "/home/b/b382291/git/one_pass"
sys.path.append(path)
os.chdir(path)

from one_pass.opa import Opa

path = "/home/b/b382291/git/AQUA/"
sys.path.append(path)
os.chdir(path)

from aqua import Reader

! hostname


l40002.lvt.dkrz.de


In [3]:
pass_dic = {"stat" : "mean", # statistic you wish to compute
"percentile_list" : None, # does not require a value unless your statistic is 'percentiles' 
"thresh_exceed" : None, # does not require a value unless your statistic is 'thresh_exceed'
"stat_freq": "daily_noon", # frequency of the statistic 
"output_freq": "monthly", # frequency that you wish to output / save the data 
"time_step": 30, # time step of the data in minutes 
"variable": "sp", # variable of interest 
"save": True, # do you want to save the final statistic 
"checkpoint": True, # do you want checkpoint each time new data is passed to the statistic (recommended yes!)
"checkpoint_filepath": "/scratch/b/b382291/data/", # path to checkpoint
"out_filepath": "/scratch/b/b382291/data/", # path to save 
}

opa_stat = Opa(pass_dic)
reader = Reader(model="ICON", exp="ngc2009", source="atm_2d_ml_R02B09")
reader.reset_stream()
data_gen = reader.retrieve(streaming_generator=True, stream_step=1, stream_unit = "hours")
count = 0 

for data in data_gen:
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")
    data = data.sp[:, 0:1000]
    dm = opa_stat.compute(data)
    count += 1 
    print('count', count)
    if count > (12 + 11*24): 
        break 

start_date: 2020-01-20T00:00:00.000000000 stop_date: 2020-01-20T00:30:00.000000000
passing on this data at 2020-01-20 00:00:00 as it is not the initial data for the requested statistic
passing on this data at 2020-01-20 00:30:00 as it is not the initial data for the requested statistic
count 1
start_date: 2020-01-20T01:00:00.000000000 stop_date: 2020-01-20T01:30:00.000000000
passing on this data at 2020-01-20 01:00:00 as it is not the initial data for the requested statistic
passing on this data at 2020-01-20 01:30:00 as it is not the initial data for the requested statistic
count 2
start_date: 2020-01-20T02:00:00.000000000 stop_date: 2020-01-20T02:30:00.000000000
passing on this data at 2020-01-20 02:00:00 as it is not the initial data for the requested statistic
passing on this data at 2020-01-20 02:30:00 as it is not the initial data for the requested statistic
count 3
start_date: 2020-01-20T03:00:00.000000000 stop_date: 2020-01-20T03:30:00.000000000
passing on this data at 2020-01-

In [5]:
dm.time

<xarray.DataArray 'time' (time: 11)>
array(['2020-01-20T13:00:00.000000000', '2020-01-21T13:00:00.000000000',
       '2020-01-22T13:00:00.000000000', '2020-01-23T13:00:00.000000000',
       '2020-01-24T13:00:00.000000000', '2020-01-25T13:00:00.000000000',
       '2020-01-26T13:00:00.000000000', '2020-01-27T13:00:00.000000000',
       '2020-01-28T13:00:00.000000000', '2020-01-29T13:00:00.000000000',
       '2020-01-30T13:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20T13:00:00 ... 2020-01-30T13:00:00
Attributes:
    axis:           T
    standard_name:  time

In [3]:
dm = getattr(dm, pass_dic["variable"] + "_"+ pass_dic["stat"])

data_arr = getattr(data, pass_dic["variable"])
ds = data_arr.isel(time=slice(n_start, n_data)) 
axNum = ds.get_axis_num('time')
np_std = np.std(ds, axis = axNum, dtype = np.float64, ddof = 1, keepdims = True) 

np.isclose(dm.values, np_std, atol = 1e-06)

#np.allclose(dm.values, np_std, atol = 1e-06)

744